In [ ]:
# Install required packages
!pip install -U bitsandbytes transformers datasets accelerate seqeval unsloth huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.6/218.6 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.2 MB/s eta 0:00:00
   ━━

In [ ]:
# Log in so we can pull gated checkpoints
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
# Clone FinEntity
!git clone https://github.com/yixuantt/FinEntity.git


Cloning into 'FinEntity'...
remote: Enumerating objects: 118, done.
remote: Total 118 (delta 0), reused 0 (delta 0), pack-reused 118 (from 1)
Receiving objects: 100% (118/118), 28.24 MiB | 15.91 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [ ]:
# Setup imports
import sys, torch, numpy as np, torch.nn as nn
sys.path.append("FinEntity")

from torch.utils.data import Dataset, random_split
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
    Trainer, TrainingArguments, DataCollatorForTokenClassification
)
from datasets import load_dataset
from seqeval.metrics import classification_report, f1_score
from sequence_aligner.labelset import LabelSet
from sequence_aligner.containers import TraingingBatch, TrainingExample

# Simple config
class config: dev_split_size = 0.2


In [ ]:
# 1) Load + reshape FinEntity
hf = load_dataset("yixuantt/FinEntity", split="train")
raw = [{"text":e["content"],
        "entities":[{"start":a["start"],"end":a["end"],"label":a["tag"]}
                    for a in e["annotations"]]}
       for e in hf]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

FinEntity.json:   0%|          | 0.00/710k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/979 [00:00<?, ? examples/s]

In [ ]:
# 2) Load gated LLaMA-3 in 4-bit
bnb = BitsAndBytesConfig(load_in_4bit=True, llm_int8_enable_fp32_cpu_offload=True)
model = AutoModelForCausalLM.from_pretrained(
    "unsloth/llama-3-8b-bnb-4bit",
    quantization_config = bnb,
    device_map          = "auto",
    use_auth_token      = True,
    trust_remote_code   = True
)
tokenizer = AutoTokenizer.from_pretrained(
    "unsloth/llama-3-8b-bnb-4bit",
    use_fast       = True,
    use_auth_token = True
)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [ ]:
# 3) Build token-classification dataset
class TrainingDatasetLLaMA(Dataset):
    def __init__(self, data, tokenizer, tpb=128):
        self.tpb       = tpb
        self.tokenizer = tokenizer
        self.label_map = {"O":0, "Positive":1, "Negative":2, "Neutral":3}
        self.examples  = []

        for ex in data:
            text, ents = ex["text"], ex["entities"]
            # 1) build char-level labels
            char_lbl = ["O"] * len(text)
            for e in ents:
                for i in range(e["start"], e["end"]):
                    if 0 <= i < len(text):
                        char_lbl[i] = e["label"]

            # 2) tokenize with offsets (no mutation!)
            toks = tokenizer(
                text,
                padding="max_length",
                truncation=True,
                max_length=tpb,
                return_attention_mask=True,
                return_offsets_mapping=True
            )
            input_ids      = toks["input_ids"]
            attention_mask = toks["attention_mask"]
            offsets        = toks["offset_mapping"]

            # 3) align token → label by zipping
            labels = []
            for (start, end), mask_bit in zip(offsets, attention_mask):
                if mask_bit == 0 or start == end:
                    labels.append(-100)
                else:
                    span = char_lbl[start:end]
                    # pick any non-O label in the span
                    lbls = set(span) - {"O"}
                    if lbls:
                        labels.append(self.label_map[lbls.pop()])
                    else:
                        labels.append(self.label_map["O"])

            # sanity: all three should be length tpb
            assert len(input_ids) == len(attention_mask) == len(labels) == tpb

            self.examples.append({
                "input_ids":      torch.tensor(input_ids,      dtype=torch.long),
                "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
                "labels":         torch.tensor(labels,         dtype=torch.long),
            })

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]



dataset = TrainingDatasetLLaMA(raw, tokenizer, tpb=128)
n       = len(dataset)
val_sz  = int(config.dev_split_size * n)   # e.g. 10% of total
train_sz = n - val_sz                      # the remaining 90%

train_ds, val_ds = random_split(dataset, [train_sz, val_sz])



In [ ]:
ex0 = train_ds[0]
print(ex0["input_ids"].shape, ex0["attention_mask"].shape, ex0["labels"].shape)
print("Unique labels:", set(ex0["labels"].tolist()))


torch.Size([128]) torch.Size([128]) torch.Size([128])
Unique labels: {0, 1, 3, -100}


In [ ]:
# 4) Define the token-classification head
class LlamaForTokenClassification(nn.Module):
    def __init__(self, base_model, hidden_size, num_labels):
        super().__init__()
        self.base       = base_model
        self.dropout    = nn.Dropout(0.1)
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        # Tell the model to return hidden states
        outputs = self.base(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True
        )
        # hidden_states is a tuple: (embeddings, layer1, ..., layerN)
        seq   = outputs.hidden_states[-1]           # last layer
        seq   = self.dropout(seq)
        logits= self.classifier(seq)                # (B, T, num_labels)

        if labels is not None:
            loss = nn.CrossEntropyLoss(ignore_index=-100)(
                logits.view(-1, logits.size(-1)),
                labels.view(-1)
            )
            return loss, logits

        return logits


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_tc = LlamaForTokenClassification(model, model.config.hidden_size, 4).to(device)
model_tc.half()


LlamaForTokenClassification(
  (base): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
          (post_att

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification
from seqeval.metrics import classification_report, f1_score
import numpy as np

# 1) Data collator (pads and uses label_pad_token_id=-100)
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True,
    label_pad_token_id=-100,
)

# 2) Metric helpers
label_inv = {0:"O",1:"Positive",2:"Negative",3:"Neutral"}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    true_labels, true_preds = [], []
    for lab_seq, pred_seq in zip(labels, preds):
        tl, tp = [], []
        for l, p in zip(lab_seq, pred_seq):
            if l == -100: continue
            tl.append(label_inv[l])
            tp.append(label_inv[p])
        true_labels.append(tl)
        true_preds.append(tp)

    report = classification_report(true_labels, true_preds, output_dict=True)
    return {
        "eval_f1": f1_score(true_labels, true_preds),
        "eval_loss": None,
        "f1_positive": report["Positive"]["f1-score"],
        "f1_negative": report["Negative"]["f1-score"],
        "f1_neutral":  report["Neutral"]["f1-score"],
        "micro_avg":   report["micro avg"]["f1-score"],
        "macro_avg":   report["macro avg"]["f1-score"],
        "weighted_avg":report["weighted avg"]["f1-score"],
    }



# 3) TrainingArguments (minimal)
training_args = TrainingArguments(
    output_dir                 = "./llama-tokenclf",
    num_train_epochs           = 3,
    per_device_train_batch_size= 8,
    per_device_eval_batch_size = 8,
    learning_rate              = 3e-5,
    logging_steps              = 50,
    logging_dir                = "./logs",
    # no evaluation_strategy here
)

# 4) Trainer
trainer = Trainer(
    model            = model_tc,
    args             = training_args,
    train_dataset    = train_ds,
    eval_dataset     = val_ds,
    data_collator    = data_collator,
    tokenizer        = tokenizer,
    compute_metrics  = compute_metrics
)

# 5) Train + Evaluate
trainer.train()
metrics = trainer.evaluate()
print("Eval metrics:", metrics)


<ipython-input-10-d7fc0d585f03>:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nv2608 (nv2608-princeton-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.947300
100,0.000000
150,0.000000
200,0.000000
250,0.000000
